# Import Libraries

In [1]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import catboost as cb
from catboost import Pool
import gc
import numpy as np
import pandas as pd
import polars as pl

---
# Load train and test datasets

In [2]:

orig_train = pl.read_csv('train_health.csv') # 351k
print("Original train shape",orig_train.shape)
display(orig_train.head())
print()

train = pl.read_csv('train.csv') # 11.5m
print("Train shape",train.shape)
display(train.head())
print()

train = pl.concat([train, orig_train]) # join both datasets together
print("Train + orig_train shape",train.shape)
display(train.head())
print()

test = pl.read_csv('test.csv') # 7m
print("Test shape",test.shape)
display(test.head())

Original train shape (381109, 12)


id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
i64,str,i64,i64,f64,i64,str,str,f64,f64,i64,i64
1,"""Male""",44,1,28.0,0,"""> 2 Years""","""Yes""",40454.0,26.0,217,1
2,"""Male""",76,1,3.0,0,"""1-2 Year""","""No""",33536.0,26.0,183,0
3,"""Male""",47,1,28.0,0,"""> 2 Years""","""Yes""",38294.0,26.0,27,1
4,"""Male""",21,1,11.0,1,"""< 1 Year""","""No""",28619.0,152.0,203,0
5,"""Female""",29,1,41.0,1,"""< 1 Year""","""No""",27496.0,152.0,39,0



Train shape (11504798, 12)


id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
i64,str,i64,i64,f64,i64,str,str,f64,f64,i64,i64
0,"""Male""",21,1,35.0,0,"""1-2 Year""","""Yes""",65101.0,124.0,187,0
1,"""Male""",43,1,28.0,0,"""> 2 Years""","""Yes""",58911.0,26.0,288,1
2,"""Female""",25,1,14.0,1,"""< 1 Year""","""No""",38043.0,152.0,254,0
3,"""Female""",35,1,1.0,0,"""1-2 Year""","""Yes""",2630.0,156.0,76,0
4,"""Female""",36,1,15.0,1,"""1-2 Year""","""No""",31951.0,152.0,294,0



Train + orig_train shape (11885907, 12)


id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
i64,str,i64,i64,f64,i64,str,str,f64,f64,i64,i64
0,"""Male""",21,1,35.0,0,"""1-2 Year""","""Yes""",65101.0,124.0,187,0
1,"""Male""",43,1,28.0,0,"""> 2 Years""","""Yes""",58911.0,26.0,288,1
2,"""Female""",25,1,14.0,1,"""< 1 Year""","""No""",38043.0,152.0,254,0
3,"""Female""",35,1,1.0,0,"""1-2 Year""","""Yes""",2630.0,156.0,76,0
4,"""Female""",36,1,15.0,1,"""1-2 Year""","""No""",31951.0,152.0,294,0



Test shape (7669866, 11)


id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
i64,str,i64,i64,f64,i64,str,str,f64,f64,i64
11504798,"""Female""",20,1,47.0,0,"""< 1 Year""","""No""",2630.0,160.0,228
11504799,"""Male""",47,1,28.0,0,"""1-2 Year""","""Yes""",37483.0,124.0,123
11504800,"""Male""",47,1,43.0,0,"""1-2 Year""","""Yes""",2630.0,26.0,271
11504801,"""Female""",22,1,47.0,1,"""< 1 Year""","""No""",24502.0,152.0,115
11504802,"""Male""",51,1,19.0,0,"""1-2 Year""","""No""",34115.0,124.0,148


---
# Add 'Response' column to test set with default value 0 for concatenation

In [3]:
target = 'Response'
test = test.with_columns(pl.lit(0).cast(pl.Int64).alias(target))

---
# Concatenate train and test datasets for preprocessing

In [4]:
df = pl.concat([train, test])

---
# Data preprocessing: Encode categorical variables and cast columns to appropriate types

In [5]:
df = df.with_columns([
    pl.col('Gender').replace({'Male': 0, 'Female': 1}).cast(pl.Int32),
    pl.col('Region_Code').cast(int),
    pl.col('Vehicle_Age').replace({'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}).cast(pl.Int32),
    pl.col('Vehicle_Damage').replace({'No': 0, 'Yes': 1}).cast(pl.Int32),
    pl.col('Annual_Premium').cast(int),
    pl.col('Policy_Sales_Channel').cast(int)
])

---
# Feature engineering: Create new features by combining existing ones and applying factorization

In [6]:
df = df.with_columns([
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Annual_Premium'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Annual_Premium'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Age'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Age'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Damage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Damage'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vintage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vintage')
])

---
# Split the processed dataframe back into train and test sets

In [7]:
train = df[:train.shape[0]].to_pandas()
test = df[train.shape[0]:].to_pandas()

---
# Prepare features and target variable for training

In [8]:
X = train.drop(['id', target], axis=1)
y = train[target]
X_test = test.drop(['id', target], axis=1)

display(y.value_counts())
display(train.head())
print()
display(test.head())

Response
0    10424138
1     1461769
Name: count, dtype: int64

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Previously_Insured_Annual_Premium,Previously_Insured_Vehicle_Age,Previously_Insured_Vehicle_Damage,Previously_Insured_Vintage
0,0,0,21,1,35,0,1,1,65101,124,187,0,0,0,0,0
1,1,0,43,1,28,0,2,1,58911,26,288,1,1,1,0,1
2,2,1,25,1,14,1,0,0,38043,152,254,0,2,2,1,2
3,3,1,35,1,1,0,1,1,2630,156,76,0,3,0,0,3
4,4,1,36,1,15,1,1,0,31951,152,294,0,4,3,1,4


,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Previously_Insured_Annual_Premium,Previously_Insured_Vehicle_Age,Previously_Insured_Vehicle_Damage,Previously_Insured_Vintage
0,11504798,1,20,1,47,0,0,0,2630,160,228,0,3,4,2,55
1,11504799,0,47,1,28,0,1,1,37483,124,123,0,1211,0,0,230
2,11504800,0,47,1,43,0,1,1,2630,26,271,0,3,0,0,392
3,11504801,1,22,1,47,1,0,0,24502,152,115,0,7102,2,1,325
4,11504802,0,51,1,19,0,1,0,34115,124,148,0,14835,0,2,415


---
# Define Stratified K-Fold cross-validation

In [9]:
FOLDS = 10
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

---
# Initialize a zero array to store predictions for the test set across all folds

In [10]:
test_preds = np.zeros((len(X_test), FOLDS), dtype=np.float32)

---
# Initialize a list to store ROC-AUC scores for each fold

In [11]:
scores = []

---
# Loop over each fold in the Stratified K-Folds cross-validator
# In each iteration:
- Extract the training and validation sets based on current fold indices.
- Create Pool objects for the training and validation sets with categorical features specified.
- Initialize the CatBoostClassifier with specified hyperparameters.
- Fit the model using the training set and evaluate it on the validation set, with early stopping.
- Record the best AUC score on the validation set for the current fold.
- Store the test set predictions for the current fold.
- Clean up memory by deleting unnecessary variables and calling garbage collection.

In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print("#" * 25)
    print(f"# Fold {fold + 1}")
    print("#" * 25)
    X_train_fold = X.loc[train_idx]
    y_train_fold = y.loc[train_idx].values
    X_val_fold = X.loc[val_idx]
    y_val_fold = y.loc[val_idx].values
    X_train_pool = Pool(X_train_fold, y_train_fold, cat_features=X.columns.values)
    X_val_pool = Pool(X_val_fold, y_val_fold, cat_features=X.columns.values)
    X_test_pool = Pool(X_test, cat_features=X_test.columns.values)




    model = cb.CatBoostClassifier(
        iterations=30000,
        learning_rate=0.02,
        random_strength=0.1,
        depth=8,
        loss_function='Logloss',
        eval_metric='AUC',
        leaf_estimation_method='Newton',
        random_state=1,
        subsample=0.9,
        bootstrap_type='Bernoulli',
        task_type='GPU'
    )

    model.fit(X=X_train_pool,
              eval_set=X_val_pool,
              verbose=500,
              early_stopping_rounds=200)

    score = model.best_score_['validation']['AUC']
    print('Fold ROC-AUC score: ', score)
    scores.append(score)

    test_preds[:, fold] = model.predict_proba(X_test)[:, 1]

    del X_train_fold, y_train_fold
    del X_val_fold, y_val_fold
    del X_train_pool, X_val_pool, X_test_pool
    del model
    gc.collect()

#########################
# Fold 1
#########################
Default metric period is 5 because AUC is/are not implemented for GPU
0:	test: 0.8735059	best: 0.8735059 (0)	total: 331ms	remaining: 2h 45m 41s
500:	test: 0.8926879	best: 0.8926879 (500)	total: 2m 50s	remaining: 2h 47m 10s
1000:	test: 0.8940567	best: 0.8940567 (1000)	total: 5m 36s	remaining: 2h 42m 24s
1500:	test: 0.8945778	best: 0.8945778 (1500)	total: 8m 22s	remaining: 2h 38m 53s
2000:	test: 0.8948709	best: 0.8948709 (2000)	total: 11m 5s	remaining: 2h 35m 7s
2500:	test: 0.8950631	best: 0.8950631 (2500)	total: 13m 48s	remaining: 2h 31m 52s
3000:	test: 0.8951964	best: 0.8951964 (3000)	total: 16m 32s	remaining: 2h 28m 52s
3500:	test: 0.8952960	best: 0.8952960 (3500)	total: 19m 14s	remaining: 2h 25m 37s
4000:	test: 0.8953860	best: 0.8953862 (3997)	total: 21m 57s	remaining: 2h 22m 40s
4500:	test: 0.8954571	best: 0.8954571 (4499)	total: 24m 40s	remaining: 2h 19m 46s
5000:	test: 0.8955115	best: 0.8955116 (4998)	total: 27m 23s	remaining: 2h 16m 53s
5500:	test: 0.8955575	best: 0.8955575 (5499)	total: 30m 5s	remaining: 2h 14m 2s
6000:	test: 0.8955936	best: 0.8955936 (5995)	total: 32m 47s	remaining: 2h 11m 7s
6500:	test: 0.8956304	best: 0.8956304 (6500)	total: 35m 30s	remaining: 2h 8m 19s
7000:	test: 0.8956588	best: 0.8956588 (6995)	total: 38m 13s	remaining: 2h 5m 34s
7500:	test: 0.8956881	best: 0.8956882 (7484)	total: 40m 56s	remaining: 2h 2m 48s
8000:	test: 0.8957099	best: 0.8957099 (8000)	total: 43m 38s	remaining: 2h
8500:	test: 0.8957320	best: 0.8957320 (8461)	total: 46m 22s	remaining: 1h 57m 15s
9000:	test: 0.8957514	best: 0.8957516 (8977)	total: 49m 7s	remaining: 1h 54m 36s
9500:	test: 0.8957758	best: 0.8957759 (9493)	total: 51m 49s	remaining: 1h 51m 49s
10000:	test: 0.8957922	best: 0.8957922 (9992)	total: 54m 32s	remaining: 1h 49m 4s
10500:	test: 0.8958052	best: 0.8958054 (10498)	total: 57m 16s	remaining: 1h 46m 21s
11000:	test: 0.8958179	best: 0.8958181 (10987)	total: 59m 59s	remaining: 1h 43m 36s
11500:	test: 0.8958268	best: 0.8958268 (11499)	total: 1h 2m 42s	remaining: 1h 40m 51s
12000:	test: 0.8958358	best: 0.8958358 (11997)	total: 1h 5m 24s	remaining: 1h 38m 6s
12500:	test: 0.8958458	best: 0.8958460 (12464)	total: 1h 8m 8s	remaining: 1h 35m 23s
13000:	test: 0.8958542	best: 0.8958546 (12988)	total: 1h 10m 50s	remaining: 1h 32m 37s
13500:	test: 0.8958617	best: 0.8958619 (13468)	total: 1h 13m 34s	remaining: 1h 29m 54s
14000:	test: 0.8958675	best: 0.8958675 (13998)	total: 1h 16m 18s	remaining: 1h 27m 11s
14500:	test: 0.8958728	best: 0.8958728 (14499)	total: 1h 19m 2s	remaining: 1h 24m 29s
15000:	test: 0.8958772	best: 0.8958772 (15000)	total: 1h 21m 46s	remaining: 1h 21m 45s
15500:	test: 0.8958827	best: 0.8958832 (15463)	total: 1h 24m 31s	remaining: 1h 19m 3s
16000:	test: 0.8958869	best: 0.8958870 (15936)	total: 1h 27m 13s	remaining: 1h 16m 19s
16500:	test: 0.8958961	best: 0.8958961 (16497)	total: 1h 29m 55s	remaining: 1h 13m 33s
17000:	test: 0.8958990	best: 0.8959000 (16964)	total: 1h 32m 39s	remaining: 1h 10m 50s
17500:	test: 0.8959033	best: 0.8959046 (17407)	total: 1h 35m 23s	remaining: 1h 8m 7s
18000:	test: 0.8959085	best: 0.8959088 (17992)	total: 1h 38m 7s	remaining: 1h 5m 24s
18500:	test: 0.8959129	best: 0.8959137 (18468)	total: 1h 40m 52s	remaining: 1h 2m 41s
19000:	test: 0.8959166	best: 0.8959168 (18990)	total: 1h 43m 35s	remaining: 59m 57s
#### bestTest = 0.8959190845
#### bestIteration = 19186
#### Shrink model to first 19187 iterations.
#### Fold ROC-AUC score:  0.8959190845489502
#########################
# Fold 2
#########################
Default metric period is 5 because AUC is/are not implemented for GPU
0:	test: 0.8744379	best: 0.8744379 (0)	total: 328ms	remaining: 2h 43m 50s
500:	test: 0.8932457	best: 0.8932457 (500)	total: 2m 51s	remaining: 2h 48m 19s
1000:	test: 0.8945486	best: 0.8945486 (1000)	total: 5m 37s	remaining: 2h 42m 44s
1500:	test: 0.8950650	best: 0.8950650 (1500)	total: 8m 20s	remaining: 2h 38m 24s
2000:	test: 0.8953688	best: 0.8953688 (1999)	total: 11m 2s	remaining: 2h 34m 35s
2500:	test: 0.8955584	best: 0.8955584 (2500)	total: 13m 45s	remaining: 2h 31m 11s
3000:	test: 0.8956999	best: 0.8956999 (3000)	total: 16m 28s	remaining: 2h 28m 12s
3500:	test: 0.8958131	best: 0.8958131 (3500)	total: 19m 11s	remaining: 2h 25m 18s
4000:	test: 0.8958879	best: 0.8958879 (4000)	total: 21m 54s	remaining: 2h 22m 24s
4500:	test: 0.8959631	best: 0.8959631 (4499)	total: 24m 38s	remaining: 2h 19m 35s
5000:	test: 0.8960201	best: 0.8960201 (5000)	total: 27m 23s	remaining: 2h 16m 53s
5500:	test: 0.8960662	best: 0.8960662 (5500)	total: 30m 5s	remaining: 2h 14m
6000:	test: 0.8961025	best: 0.8961025 (6000)	total: 32m 45s	remaining: 2h 11m 1s
6500:	test: 0.8961375	best: 0.8961375 (6490)	total: 35m 27s	remaining: 2h 8m 10s
7000:	test: 0.8961684	best: 0.8961684 (7000)	total: 38m 11s	remaining: 2h 5m 28s
7500:	test: 0.8961930	best: 0.8961931 (7488)	total: 40m 54s	remaining: 2h 2m 42s
8000:	test: 0.8962166	best: 0.8962166 (8000)	total: 43m 37s	remaining: 1h 59m 57s
8500:	test: 0.8962358	best: 0.8962358 (8496)	total: 46m 18s	remaining: 1h 57m 7s
9000:	test: 0.8962519	best: 0.8962519 (9000)	total: 49m 1s	remaining: 1h 54m 22s
9500:	test: 0.8962695	best: 0.8962698 (9491)	total: 51m 45s	remaining: 1h 51m 40s
10000:	test: 0.8962879	best: 0.8962880 (9999)	total: 54m 28s	remaining: 1h 48m 55s
10500:	test: 0.8963005	best: 0.8963005 (10499)	total: 57m 11s	remaining: 1h 46m 11s
11000:	test: 0.8963131	best: 0.8963131 (11000)	total: 59m 53s	remaining: 1h 43m 26s
11500:	test: 0.8963273	best: 0.8963274 (11499)	total: 1h 2m 36s	remaining: 1h 40m 42s
12000:	test: 0.8963387	best: 0.8963387 (12000)	total: 1h 5m 21s	remaining: 1h 38m 1s
12500:	test: 0.8963472	best: 0.8963476 (12481)	total: 1h 8m 3s	remaining: 1h 35m 16s
13000:	test: 0.8963529	best: 0.8963532 (12958)	total: 1h 10m 47s	remaining: 1h 32m 34s
13500:	test: 0.8963619	best: 0.8963623 (13495)	total: 1h 13m 31s	remaining: 1h 29m 50s
14000:	test: 0.8963671	best: 0.8963671 (13984)	total: 1h 16m 15s	remaining: 1h 27m 8s
14500:	test: 0.8963749	best: 0.8963750 (14498)	total: 1h 18m 59s	remaining: 1h 24m 25s
15000:	test: 0.8963780	best: 0.8963782 (14989)	total: 1h 21m 43s	remaining: 1h 21m 42s
15500:	test: 0.8963812	best: 0.8963818 (15476)	total: 1h 24m 28s	remaining: 1h 19m
16000:	test: 0.8963853	best: 0.8963856 (15985)	total: 1h 27m 12s	remaining: 1h 16m 18s
16500:	test: 0.8963905	best: 0.8963907 (16494)	total: 1h 29m 55s	remaining: 1h 13m 33s
17000:	test: 0.8963957	best: 0.8963962 (16977)	total: 1h 32m 41s	remaining: 1h 10m 51s
17500:	test: 0.8963989	best: 0.8963996 (17418)	total: 1h 35m 26s	remaining: 1h 8m 9s
18000:	test: 0.8964026	best: 0.8964038 (17882)	total: 1h 38m 11s	remaining: 1h 5m 27s
18500:	test: 0.8964053	best: 0.8964061 (18456)	total: 1h 40m 55s	remaining: 1h 2m 43s
19000:	test: 0.8964104	best: 0.8964105 (18991)	total: 1h 43m 39s	remaining: 1h
19500:	test: 0.8964130	best: 0.8964134 (19431)	total: 1h 46m 24s	remaining: 57m 17s
#### bestTest = 0.8964133859
#### bestIteration = 19431
#### Shrink model to first 19432 iterations.
#### Fold ROC-AUC score:  0.8964133858680725
#########################
# Fold 3
#########################
Default metric period is 5 because AUC is/are not implemented for GPU
0:	test: 0.8729446	best: 0.8729446 (0)	total: 330ms	remaining: 2h 45m 11s
500:	test: 0.8920310	best: 0.8920310 (500)	total: 2m 50s	remaining: 2h 47m 41s
1000:	test: 0.8934515	best: 0.8934515 (1000)	total: 5m 36s	remaining: 2h 42m 31s
1500:	test: 0.8939962	best: 0.8939962 (1500)	total: 8m 20s	remaining: 2h 38m 30s
2000:	test: 0.8942999	best: 0.8942999 (2000)	total: 11m 3s	remaining: 2h 34m 38s
2500:	test: 0.8945103	best: 0.8945103 (2500)	total: 13m 44s	remaining: 2h 31m 2s
3000:	test: 0.8946510	best: 0.8946510 (3000)	total: 16m 26s	remaining: 2h 27m 58s
3500:	test: 0.8947546	best: 0.8947546 (3500)	total: 19m 9s	remaining: 2h 24m 58s
4000:	test: 0.8948452	best: 0.8948452 (4000)	total: 21m 51s	remaining: 2h 22m 5s
4500:	test: 0.8949233	best: 0.8949233 (4500)	total: 24m 33s	remaining: 2h 19m 10s
5000:	test: 0.8949848	best: 0.8949848 (5000)	total: 27m 17s	remaining: 2h 16m 23s
5500:	test: 0.8950383	best: 0.8950383 (5500)	total: 30m	remaining: 2h 13m 39s
6000:	test: 0.8950786	best: 0.8950786 (5999)	total: 32m 44s	remaining: 2h 10m 54s
6500:	test: 0.8951191	best: 0.8951192 (6499)	total: 35m 27s	remaining: 2h 8m 9s
7000:	test: 0.8951545	best: 0.8951547 (6995)	total: 38m 11s	remaining: 2h 5m 26s
7500:	test: 0.8951816	best: 0.8951816 (7498)	total: 40m 54s	remaining: 2h 2m 42s
8000:	test: 0.8952080	best: 0.8952080 (8000)	total: 43m 36s	remaining: 1h 59m 52s
8500:	test: 0.8952316	best: 0.8952316 (8500)	total: 46m 18s	remaining: 1h 57m 6s
9000:	test: 0.8952554	best: 0.8952554 (8999)	total: 49m 1s	remaining: 1h 54m 21s
9500:	test: 0.8952772	best: 0.8952776 (9491)	total: 51m 44s	remaining: 1h 51m 37s
10000:	test: 0.8952911	best: 0.8952915 (9990)	total: 54m 29s	remaining: 1h 48m 57s
10500:	test: 0.8953077	best: 0.8953078 (10494)	total: 57m 10s	remaining: 1h 46m 10s
11000:	test: 0.8953190	best: 0.8953191 (10996)	total: 59m 55s	remaining: 1h 43m 28s
11500:	test: 0.8953310	best: 0.8953310 (11500)	total: 1h 2m 36s	remaining: 1h 40m 42s
12000:	test: 0.8953444	best: 0.8953444 (11998)	total: 1h 5m 19s	remaining: 1h 37m 58s
12500:	test: 0.8953567	best: 0.8953567 (12500)	total: 1h 8m 2s	remaining: 1h 35m 14s
13000:	test: 0.8953655	best: 0.8953656 (12994)	total: 1h 10m 46s	remaining: 1h 32m 32s
13500:	test: 0.8953746	best: 0.8953746 (13491)	total: 1h 13m 29s	remaining: 1h 29m 49s
14000:	test: 0.8953808	best: 0.8953808 (14000)	total: 1h 16m 14s	remaining: 1h 27m 6s
14500:	test: 0.8953884	best: 0.8953885 (14487)	total: 1h 18m 57s	remaining: 1h 24m 23s
15000:	test: 0.8953983	best: 0.8953983 (14999)	total: 1h 21m 41s	remaining: 1h 21m 40s
15500:	test: 0.8954033	best: 0.8954034 (15493)	total: 1h 24m 23s	remaining: 1h 18m 56s
16000:	test: 0.8954108	best: 0.8954110 (15992)	total: 1h 27m 7s	remaining: 1h 16m 13s
16500:	test: 0.8954182	best: 0.8954182 (16500)	total: 1h 29m 50s	remaining: 1h 13m 29s
17000:	test: 0.8954237	best: 0.8954241 (16994)	total: 1h 32m 34s	remaining: 1h 10m 46s
17500:	test: 0.8954296	best: 0.8954297 (17495)	total: 1h 35m 18s	remaining: 1h 8m 4s
#### bestTest = 0.8954312801
#### bestIteration = 17659
#### Shrink model to first 17660 iterations.
#### Fold ROC-AUC score:  0.8954312801361084
#########################
# Fold 4
#########################
Default metric period is 5 because AUC is/are not implemented for GPU
0:	test: 0.8724163	best: 0.8724163 (0)	total: 365ms	remaining: 3h 2m 19s
500:	test: 0.8929229	best: 0.8929229 (500)	total: 2m 50s	remaining: 2h 47m 42s
1000:	test: 0.8942332	best: 0.8942332 (1000)	total: 5m 36s	remaining: 2h 42m 17s
1500:	test: 0.8947297	best: 0.8947297 (1500)	total: 8m 19s	remaining: 2h 38m 1s
2000:	test: 0.8950347	best: 0.8950347 (2000)	total: 11m	remaining: 2h 34m
2500:	test: 0.8952182	best: 0.8952182 (2500)	total: 13m 42s	remaining: 2h 30m 45s
3000:	test: 0.8953544	best: 0.8953544 (3000)	total: 16m 25s	remaining: 2h 27m 43s
3500:	test: 0.8954539	best: 0.8954539 (3500)	total: 19m 6s	remaining: 2h 24m 39s
4000:	test: 0.8955367	best: 0.8955367 (4000)	total: 21m 49s	remaining: 2h 21m 47s
4500:	test: 0.8956027	best: 0.8956027 (4500)	total: 24m 31s	remaining: 2h 18m 56s
5000:	test: 0.8956598	best: 0.8956598 (5000)	total: 27m 14s	remaining: 2h 16m 11s
5500:	test: 0.8957077	best: 0.8957077 (5498)	total: 29m 57s	remaining: 2h 13m 26s
6000:	test: 0.8957462	best: 0.8957463 (5998)	total: 32m 40s	remaining: 2h 10m 40s
6500:	test: 0.8957796	best: 0.8957796 (6498)	total: 35m 23s	remaining: 2h 7m 55s
7000:	test: 0.8958090	best: 0.8958092 (6988)	total: 38m 6s	remaining: 2h 5m 11s
7500:	test: 0.8958323	best: 0.8958325 (7496)	total: 40m 49s	remaining: 2h 2m 26s
8000:	test: 0.8958531	best: 0.8958531 (8000)	total: 43m 32s	remaining: 1h 59m 42s
8500:	test: 0.8958743	best: 0.8958743 (8500)	total: 46m 14s	remaining: 1h 56m 55s
9000:	test: 0.8958937	best: 0.8958937 (8996)	total: 48m 56s	remaining: 1h 54m 11s
9500:	test: 0.8959113	best: 0.8959114 (9499)	total: 51m 40s	remaining: 1h 51m 28s
10000:	test: 0.8959244	best: 0.8959247 (9999)	total: 54m 22s	remaining: 1h 48m 43s
10500:	test: 0.8959416	best: 0.8959417 (10498)	total: 57m 6s	remaining: 1h 46m 2s
11000:	test: 0.8959534	best: 0.8959534 (10999)	total: 59m 49s	remaining: 1h 43m 18s
11500:	test: 0.8959633	best: 0.8959633 (11500)	total: 1h 2m 33s	remaining: 1h 40m 37s
12000:	test: 0.8959721	best: 0.8959721 (11999)	total: 1h 5m 18s	remaining: 1h 37m 56s
12500:	test: 0.8959801	best: 0.8959803 (12497)	total: 1h 8m 3s	remaining: 1h 35m 16s
13000:	test: 0.8959888	best: 0.8959890 (12993)	total: 1h 10m 46s	remaining: 1h 32m 32s
13500:	test: 0.8959969	best: 0.8959969 (13500)	total: 1h 13m 29s	remaining: 1h 29m 48s
14000:	test: 0.8960047	best: 0.8960051 (13993)	total: 1h 16m 12s	remaining: 1h 27m 4s
14500:	test: 0.8960106	best: 0.8960108 (14493)	total: 1h 18m 55s	remaining: 1h 24m 21s
15000:	test: 0.8960164	best: 0.8960165 (14995)	total: 1h 21m 40s	remaining: 1h 21m 39s
15500:	test: 0.8960201	best: 0.8960201 (15500)	total: 1h 24m 22s	remaining: 1h 18m 55s
#### bestTest = 0.8960219026
#### bestIteration = 15656
#### Shrink model to first 15657 iterations.
#### Fold ROC-AUC score:  0.8960219025611877
#########################
# Fold 5
#########################
Default metric period is 5 because AUC is/are not implemented for GPU
0:	test: 0.8739901	best: 0.8739901 (0)	total: 301ms	remaining: 2h 30m 15s
500:	test: 0.8925250	best: 0.8925250 (500)	total: 2m 49s	remaining: 2h 46m 17s
1000:	test: 0.8938118	best: 0.8938118 (1000)	total: 5m 33s	remaining: 2h 40m 58s
1500:	test: 0.8943073	best: 0.8943073 (1500)	total: 8m 17s	remaining: 2h 37m 25s
2000:	test: 0.8946031	best: 0.8946031 (2000)	total: 10m 59s	remaining: 2h 33m 51s
2500:	test: 0.8947927	best: 0.8947928 (2499)	total: 13m 41s	remaining: 2h 30m 36s
3000:	test: 0.8949299	best: 0.8949299 (3000)	total: 16m 23s	remaining: 2h 27m 30s
3500:	test: 0.8950303	best: 0.8950303 (3499)	total: 19m 6s	remaining: 2h 24m 41s
4000:	test: 0.8951116	best: 0.8951116 (4000)	total: 21m 48s	remaining: 2h 21m 43s
4500:	test: 0.8951811	best: 0.8951812 (4499)	total: 24m 30s	remaining: 2h 18m 51s
5000:	test: 0.8952391	best: 0.8952391 (5000)	total: 27m 13s	remaining: 2h 16m 5s
5500:	test: 0.8952862	best: 0.8952862 (5497)	total: 29m 54s	remaining: 2h 13m 11s
6000:	test: 0.8953271	best: 0.8953271 (6000)	total: 32m 36s	remaining: 2h 10m 25s
6500:	test: 0.8953539	best: 0.8953539 (6500)	total: 35m 19s	remaining: 2h 7m 41s
7000:	test: 0.8953862	best: 0.8953862 (7000)	total: 38m 1s	remaining: 2h 4m 54s
7500:	test: 0.8954094	best: 0.8954095 (7499)	total: 40m 43s	remaining: 2h 2m 9s
8000:	test: 0.8954297	best: 0.8954297 (7993)	total: 43m 25s	remaining: 1h 59m 23s
8500:	test: 0.8954474	best: 0.8954474 (8500)	total: 46m 8s	remaining: 1h 56m 42s
9000:	test: 0.8954687	best: 0.8954691 (8981)	total: 48m 53s	remaining: 1h 54m 3s
9500:	test: 0.8954856	best: 0.8954856 (9497)	total: 51m 35s	remaining: 1h 51m 18s
10000:	test: 0.8954982	best: 0.8954982 (10000)	total: 54m 18s	remaining: 1h 48m 35s
10500:	test: 0.8955093	best: 0.8955094 (10496)	total: 57m	remaining: 1h 45m 52s
11000:	test: 0.8955249	best: 0.8955249 (10999)	total: 59m 43s	remaining: 1h 43m 9s
11500:	test: 0.8955345	best: 0.8955346 (11499)	total: 1h 2m 27s	remaining: 1h 40m 27s
12000:	test: 0.8955448	best: 0.8955449 (11997)	total: 1h 5m 10s	remaining: 1h 37m 44s
12500:	test: 0.8955575	best: 0.8955575 (12499)	total: 1h 7m 54s	remaining: 1h 35m 3s
13000:	test: 0.8955629	best: 0.8955630 (12986)	total: 1h 10m 35s	remaining: 1h 32m 18s
13500:	test: 0.8955708	best: 0.8955708 (13500)	total: 1h 13m 18s	remaining: 1h 29m 35s
14000:	test: 0.8955821	best: 0.8955821 (13995)	total: 1h 16m 2s	remaining: 1h 26m 53s
14500:	test: 0.8955884	best: 0.8955891 (14438)	total: 1h 18m 45s	remaining: 1h 24m 11s
15000:	test: 0.8955948	best: 0.8955957 (14898)	total: 1h 21m 28s	remaining: 1h 21m 28s
15500:	test: 0.8955992	best: 0.8955995 (15453)	total: 1h 24m 11s	remaining: 1h 18m 45s
16000:	test: 0.8956028	best: 0.8956033 (15980)	total: 1h 26m 56s	remaining: 1h 16m 3s
#### bestTest = 0.895603478
#### bestIteration = 16038
#### Shrink model to first 16039 iterations.
#### Fold ROC-AUC score:  0.8956034779548645
#########################
# Fold 6
#########################
Default metric period is 5 because AUC is/are not implemented for GPU
0:	test: 0.8734017	best: 0.8734017 (0)	total: 325ms	remaining: 2h 42m 17s
500:	test: 0.8922227	best: 0.8922227 (500)	total: 2m 49s	remaining: 2h 46m 21s
1000:	test: 0.8935996	best: 0.8935996 (1000)	total: 5m 34s	remaining: 2h 41m 31s
1500:	test: 0.8941075	best: 0.8941075 (1500)	total: 8m 17s	remaining: 2h 37m 27s
2000:	test: 0.8944245	best: 0.8944245 (2000)	total: 10m 59s	remaining: 2h 33m 54s
2500:	test: 0.8946331	best: 0.8946331 (2500)	total: 13m 43s	remaining: 2h 30m 54s
3000:	test: 0.8947741	best: 0.8947741 (3000)	total: 16m 26s	remaining: 2h 27m 51s
3500:	test: 0.8948772	best: 0.8948772 (3500)	total: 19m 9s	remaining: 2h 25m 3s
4000:	test: 0.8949660	best: 0.8949660 (4000)	total: 21m 53s	remaining: 2h 22m 17s
4500:	test: 0.8950399	best: 0.8950399 (4500)	total: 24m 36s	remaining: 2h 19m 24s
5000:	test: 0.8951016	best: 0.8951016 (5000)	total: 27m 19s	remaining: 2h 16m 35s
5500:	test: 0.8951470	best: 0.8951470 (5500)	total: 30m 2s	remaining: 2h 13m 48s
6000:	test: 0.8951849	best: 0.8951849 (6000)	total: 32m 45s	remaining: 2h 11m
6500:	test: 0.8952201	best: 0.8952201 (6498)	total: 35m 29s	remaining: 2h 8m 17s
7000:	test: 0.8952509	best: 0.8952509 (6999)	total: 38m 12s	remaining: 2h 5m 32s
7500:	test: 0.8952777	best: 0.8952778 (7496)	total: 40m 56s	remaining: 2h 2m 47s
8000:	test: 0.8953053	best: 0.8953055 (7998)	total: 43m 39s	remaining: 2h 1s
8500:	test: 0.8953238	best: 0.8953238 (8491)	total: 46m 21s	remaining: 1h 57m 15s
9000:	test: 0.8953405	best: 0.8953409 (8998)	total: 49m 4s	remaining: 1h 54m 29s
9500:	test: 0.8953589	best: 0.8953589 (9500)	total: 51m 48s	remaining: 1h 51m 47s
10000:	test: 0.8953721	best: 0.8953722 (9999)	total: 54m 32s	remaining: 1h 49m 4s
10500:	test: 0.8953864	best: 0.8953865 (10497)	total: 57m 15s	remaining: 1h 46m 19s
11000:	test: 0.8953990	best: 0.8953990 (10990)	total: 59m 58s	remaining: 1h 43m 34s
11500:	test: 0.8954105	best: 0.8954108 (11481)	total: 1h 2m 41s	remaining: 1h 40m 50s
12000:	test: 0.8954225	best: 0.8954225 (11997)	total: 1h 5m 25s	remaining: 1h 38m 7s
12500:	test: 0.8954282	best: 0.8954287 (12416)	total: 1h 8m 8s	remaining: 1h 35m 23s
13000:	test: 0.8954372	best: 0.8954372 (12999)	total: 1h 10m 51s	remaining: 1h 32m 39s
13500:	test: 0.8954467	best: 0.8954468 (13498)	total: 1h 13m 36s	remaining: 1h 29m 57s
14000:	test: 0.8954516	best: 0.8954517 (13999)	total: 1h 16m 19s	remaining: 1h 27m 12s
14500:	test: 0.8954574	best: 0.8954579 (14445)	total: 1h 19m 2s	remaining: 1h 24m 28s
15000:	test: 0.8954630	best: 0.8954635 (14988)	total: 1h 21m 45s	remaining: 1h 21m 44s
15500:	test: 0.8954685	best: 0.8954686 (15489)	total: 1h 24m 28s	remaining: 1h 19m
16000:	test: 0.8954761	best: 0.8954764 (15963)	total: 1h 27m 12s	remaining: 1h 16m 18s
16500:	test: 0.8954814	best: 0.8954818 (16462)	total: 1h 29m 56s	remaining: 1h 13m 35s
17000:	test: 0.8954870	best: 0.8954872 (16969)	total: 1h 32m 41s	remaining: 1h 10m 52s
17500:	test: 0.8954933	best: 0.8954933 (17481)	total: 1h 35m 24s	remaining: 1h 8m 8s
18000:	test: 0.8954974	best: 0.8954980 (17964)	total: 1h 38m 9s	remaining: 1h 5m 25s
#### bestTest = 0.8954980373
#### bestIteration = 18125
#### Shrink model to first 18126 iterations.
#### Fold ROC-AUC score:  0.8954980373382568
#########################
# Fold 7
#########################
Default metric period is 5 because AUC is/are not implemented for GPU
0:	test: 0.8735401	best: 0.8735401 (0)	total: 298ms	remaining: 2h 29m 9s
500:	test: 0.8922106	best: 0.8922106 (500)	total: 2m 51s	remaining: 2h 48m 26s
1000:	test: 0.8935765	best: 0.8935765 (1000)	total: 5m 37s	remaining: 2h 42m 48s
1500:	test: 0.8941154	best: 0.8941154 (1500)	total: 8m 20s	remaining: 2h 38m 16s
2000:	test: 0.8944448	best: 0.8944448 (2000)	total: 11m 2s	remaining: 2h 34m 32s
2500:	test: 0.8946482	best: 0.8946482 (2499)	total: 13m 45s	remaining: 2h 31m 20s
3000:	test: 0.8948002	best: 0.8948002 (3000)	total: 16m 27s	remaining: 2h 28m 7s
3500:	test: 0.8949126	best: 0.8949126 (3500)	total: 19m 10s	remaining: 2h 25m 5s
4000:	test: 0.8949991	best: 0.8949991 (4000)	total: 21m 52s	remaining: 2h 22m 7s
4500:	test: 0.8950655	best: 0.8950655 (4500)	total: 24m 34s	remaining: 2h 19m 14s
5000:	test: 0.8951253	best: 0.8951254 (4992)	total: 27m 16s	remaining: 2h 16m 21s
5500:	test: 0.8951733	best: 0.8951733 (5500)	total: 29m 59s	remaining: 2h 13m 35s
6000:	test: 0.8952173	best: 0.8952173 (5997)	total: 32m 41s	remaining: 2h 10m 44s
6500:	test: 0.8952506	best: 0.8952506 (6500)	total: 35m 27s	remaining: 2h 8m 9s
7000:	test: 0.8952814	best: 0.8952814 (6998)	total: 38m 10s	remaining: 2h 5m 23s
7500:	test: 0.8953082	best: 0.8953083 (7499)	total: 40m 53s	remaining: 2h 2m 38s
8000:	test: 0.8953273	best: 0.8953274 (7995)	total: 43m 36s	remaining: 1h 59m 52s
8500:	test: 0.8953517	best: 0.8953517 (8500)	total: 46m 19s	remaining: 1h 57m 9s
9000:	test: 0.8953692	best: 0.8953694 (8998)	total: 49m 2s	remaining: 1h 54m 25s
9500:	test: 0.8953884	best: 0.8953884 (9500)	total: 51m 47s	remaining: 1h 51m 44s
10000:	test: 0.8954024	best: 0.8954027 (9987)	total: 54m 32s	remaining: 1h 49m 4s
10500:	test: 0.8954141	best: 0.8954143 (10494)	total: 57m 16s	remaining: 1h 46m 20s
11000:	test: 0.8954291	best: 0.8954291 (11000)	total: 1h	remaining: 1h 43m 38s
11500:	test: 0.8954394	best: 0.8954395 (11483)	total: 1h 2m 44s	remaining: 1h 40m 55s
12000:	test: 0.8954496	best: 0.8954498 (11995)	total: 1h 5m 29s	remaining: 1h 38m 13s
12500:	test: 0.8954593	best: 0.8954593 (12495)	total: 1h 8m 13s	remaining: 1h 35m 30s
13000:	test: 0.8954692	best: 0.8954695 (12993)	total: 1h 10m 57s	remaining: 1h 32m 46s
13500:	test: 0.8954781	best: 0.8954783 (13485)	total: 1h 13m 40s	remaining: 1h 30m 2s
14000:	test: 0.8954857	best: 0.8954858 (13983)	total: 1h 16m 25s	remaining: 1h 27m 19s
#### bestTest = 0.8954868317
#### bestIteration = 14044
#### Shrink model to first 14045 iterations.
#### Fold ROC-AUC score:  0.8954868316650391
#########################
# Fold 8
#########################
Default metric period is 5 because AUC is/are not implemented for GPU
0:	test: 0.8726272	best: 0.8726272 (0)	total: 356ms	remaining: 2h 57m 54s
500:	test: 0.8922919	best: 0.8922919 (500)	total: 2m 50s	remaining: 2h 47m 28s
1000:	test: 0.8936779	best: 0.8936779 (1000)	total: 5m 36s	remaining: 2h 42m 28s
1500:	test: 0.8941880	best: 0.8941880 (1500)	total: 8m 20s	remaining: 2h 38m 20s
2000:	test: 0.8944800	best: 0.8944800 (2000)	total: 11m 2s	remaining: 2h 34m 33s
2500:	test: 0.8946787	best: 0.8946787 (2500)	total: 13m 44s	remaining: 2h 31m 8s
3000:	test: 0.8948188	best: 0.8948188 (3000)	total: 16m 26s	remaining: 2h 27m 56s
3500:	test: 0.8949270	best: 0.8949270 (3500)	total: 19m 8s	remaining: 2h 24m 56s
4000:	test: 0.8950148	best: 0.8950148 (3999)	total: 21m 50s	remaining: 2h 21m 58s
4500:	test: 0.8950782	best: 0.8950782 (4500)	total: 24m 33s	remaining: 2h 19m 9s
5000:	test: 0.8951251	best: 0.8951251 (5000)	total: 27m 15s	remaining: 2h 16m 13s
5500:	test: 0.8951622	best: 0.8951624 (5492)	total: 29m 57s	remaining: 2h 13m 25s
6000:	test: 0.8952060	best: 0.8952061 (5999)	total: 32m 38s	remaining: 2h 10m 33s
6500:	test: 0.8952407	best: 0.8952407 (6499)	total: 35m 20s	remaining: 2h 7m 45s
7000:	test: 0.8952667	best: 0.8952667 (6978)	total: 38m 3s	remaining: 2h 5m 1s
7500:	test: 0.8952940	best: 0.8952941 (7499)	total: 40m 45s	remaining: 2h 2m 15s
8000:	test: 0.8953167	best: 0.8953168 (7989)	total: 43m 28s	remaining: 1h 59m 32s
8500:	test: 0.8953393	best: 0.8953395 (8498)	total: 46m 10s	remaining: 1h 56m 47s
9000:	test: 0.8953557	best: 0.8953557 (9000)	total: 48m 53s	remaining: 1h 54m 4s
9500:	test: 0.8953736	best: 0.8953736 (9498)	total: 51m 36s	remaining: 1h 51m 20s
10000:	test: 0.8953895	best: 0.8953895 (10000)	total: 54m 19s	remaining: 1h 48m 37s
10500:	test: 0.8954026	best: 0.8954026 (10500)	total: 57m 1s	remaining: 1h 45m 53s
11000:	test: 0.8954117	best: 0.8954120 (10993)	total: 59m 44s	remaining: 1h 43m 11s
11500:	test: 0.8954237	best: 0.8954239 (11495)	total: 1h 2m 29s	remaining: 1h 40m 30s
12000:	test: 0.8954324	best: 0.8954324 (11998)	total: 1h 5m 12s	remaining: 1h 37m 47s
12500:	test: 0.8954413	best: 0.8954418 (12464)	total: 1h 7m 55s	remaining: 1h 35m 4s
13000:	test: 0.8954497	best: 0.8954498 (12995)	total: 1h 10m 37s	remaining: 1h 32m 19s
13500:	test: 0.8954586	best: 0.8954586 (13500)	total: 1h 13m 19s	remaining: 1h 29m 36s
14000:	test: 0.8954681	best: 0.8954681 (14000)	total: 1h 16m 3s	remaining: 1h 26m 54s
14500:	test: 0.8954746	best: 0.8954746 (14497)	total: 1h 18m 46s	remaining: 1h 24m 11s
#### bestTest = 0.8954750299
#### bestIteration = 14509
#### Shrink model to first 14510 iterations.
#### Fold ROC-AUC score:  0.8954750299453735
#########################
# Fold 9
#########################
Default metric period is 5 because AUC is/are not implemented for GPU
0:	test: 0.8737656	best: 0.8737656 (0)	total: 336ms	remaining: 2h 48m 8s
500:	test: 0.8924196	best: 0.8924196 (500)	total: 2m 50s	remaining: 2h 47m 20s
1000:	test: 0.8937355	best: 0.8937355 (1000)	total: 5m 35s	remaining: 2h 41m 51s
1500:	test: 0.8942143	best: 0.8942143 (1500)	total: 8m 17s	remaining: 2h 37m 32s
2000:	test: 0.8944902	best: 0.8944902 (1999)	total: 10m 59s	remaining: 2h 33m 42s
2500:	test: 0.8946811	best: 0.8946811 (2500)	total: 13m 40s	remaining: 2h 30m 17s
3000:	test: 0.8948072	best: 0.8948072 (3000)	total: 16m 21s	remaining: 2h 27m 10s
3500:	test: 0.8949078	best: 0.8949079 (3498)	total: 19m 2s	remaining: 2h 24m 9s
4000:	test: 0.8949860	best: 0.8949860 (4000)	total: 21m 44s	remaining: 2h 21m 19s
4500:	test: 0.8950532	best: 0.8950532 (4498)	total: 24m 27s	remaining: 2h 18m 35s
5000:	test: 0.8951029	best: 0.8951030 (4999)	total: 27m 10s	remaining: 2h 15m 50s
5500:	test: 0.8951481	best: 0.8951481 (5500)	total: 29m 52s	remaining: 2h 13m 3s
6000:	test: 0.8951882	best: 0.8951882 (6000)	total: 32m 35s	remaining: 2h 10m 18s
6500:	test: 0.8952201	best: 0.8952202 (6499)	total: 35m 17s	remaining: 2h 7m 33s
7000:	test: 0.8952506	best: 0.8952506 (7000)	total: 37m 59s	remaining: 2h 4m 46s
7500:	test: 0.8952739	best: 0.8952740 (7499)	total: 40m 41s	remaining: 2h 2m 2s
8000:	test: 0.8952931	best: 0.8952931 (8000)	total: 43m 25s	remaining: 1h 59m 22s
8500:	test: 0.8953165	best: 0.8953165 (8500)	total: 46m 7s	remaining: 1h 56m 39s
9000:	test: 0.8953332	best: 0.8953333 (8993)	total: 48m 50s	remaining: 1h 53m 57s
9500:	test: 0.8953489	best: 0.8953489 (9499)	total: 51m 33s	remaining: 1h 51m 14s
10000:	test: 0.8953630	best: 0.8953632 (9997)	total: 54m 17s	remaining: 1h 48m 34s
10500:	test: 0.8953767	best: 0.8953767 (10497)	total: 57m	remaining: 1h 45m 51s
11000:	test: 0.8953874	best: 0.8953874 (11000)	total: 59m 43s	remaining: 1h 43m 9s
11500:	test: 0.8953997	best: 0.8953997 (11496)	total: 1h 2m 28s	remaining: 1h 40m 28s
12000:	test: 0.8954083	best: 0.8954083 (11999)	total: 1h 5m 12s	remaining: 1h 37m 47s
12500:	test: 0.8954197	best: 0.8954200 (12456)	total: 1h 7m 54s	remaining: 1h 35m 3s
13000:	test: 0.8954266	best: 0.8954268 (12991)	total: 1h 10m 37s	remaining: 1h 32m 20s
13500:	test: 0.8954325	best: 0.8954325 (13500)	total: 1h 13m 20s	remaining: 1h 29m 37s
14000:	test: 0.8954371	best: 0.8954374 (13983)	total: 1h 16m 4s	remaining: 1h 26m 55s
14500:	test: 0.8954440	best: 0.8954442 (14495)	total: 1h 18m 48s	remaining: 1h 24m 13s
15000:	test: 0.8954486	best: 0.8954492 (14955)	total: 1h 21m 31s	remaining: 1h 21m 30s
15500:	test: 0.8954571	best: 0.8954573 (15478)	total: 1h 24m 15s	remaining: 1h 18m 48s
#### bestTest = 0.8954583108
#### bestIteration = 15585
#### Shrink model to first 15586 iterations.
#### Fold ROC-AUC score:  0.895458310842514
#########################
# Fold 10
#########################
Default metric period is 5 because AUC is/are not implemented for GPU
0:	test: 0.8730494	best: 0.8730494 (0)	total: 363ms	remaining: 3h 1m 32s
500:	test: 0.8925238	best: 0.8925238 (500)	total: 2m 49s	remaining: 2h 46m 32s
1000:	test: 0.8938801	best: 0.8938801 (1000)	total: 5m 34s	remaining: 2h 41m 21s
1500:	test: 0.8943878	best: 0.8943878 (1500)	total: 8m 16s	remaining: 2h 37m 15s
2000:	test: 0.8946787	best: 0.8946787 (2000)	total: 10m 59s	remaining: 2h 33m 47s
2500:	test: 0.8948646	best: 0.8948646 (2500)	total: 13m 41s	remaining: 2h 30m 32s
3000:	test: 0.8949975	best: 0.8949975 (3000)	total: 16m 23s	remaining: 2h 27m 29s
3500:	test: 0.8950949	best: 0.8950949 (3500)	total: 19m 6s	remaining: 2h 24m 37s
4000:	test: 0.8951743	best: 0.8951743 (3999)	total: 21m 50s	remaining: 2h 21m 55s
4500:	test: 0.8952340	best: 0.8952340 (4500)	total: 24m 32s	remaining: 2h 19m 1s
5000:	test: 0.8952888	best: 0.8952889 (4999)	total: 27m 14s	remaining: 2h 16m 8s
5500:	test: 0.8953341	best: 0.8953341 (5500)	total: 29m 55s	remaining: 2h 13m 15s
6000:	test: 0.8953749	best: 0.8953749 (5999)	total: 32m 38s	remaining: 2h 10m 31s
6500:	test: 0.8954009	best: 0.8954009 (6494)	total: 35m 21s	remaining: 2h 7m 48s
7000:	test: 0.8954303	best: 0.8954306 (6992)	total: 38m 4s	remaining: 2h 5m 5s
7500:	test: 0.8954516	best: 0.8954516 (7499)	total: 40m 49s	remaining: 2h 2m 25s
8000:	test: 0.8954715	best: 0.8954715 (8000)	total: 43m 33s	remaining: 1h 59m 45s
8500:	test: 0.8954859	best: 0.8954861 (8484)	total: 46m 15s	remaining: 1h 57m
9000:	test: 0.8955027	best: 0.8955027 (9000)	total: 48m 57s	remaining: 1h 54m 12s
9500:	test: 0.8955196	best: 0.8955197 (9494)	total: 51m 40s	remaining: 1h 51m 29s
10000:	test: 0.8955336	best: 0.8955336 (10000)	total: 54m 25s	remaining: 1h 48m 49s
10500:	test: 0.8955446	best: 0.8955448 (10487)	total: 57m 7s	remaining: 1h 46m 5s
11000:	test: 0.8955548	best: 0.8955549 (10999)	total: 59m 51s	remaining: 1h 43m 22s
11500:	test: 0.8955654	best: 0.8955654 (11500)	total: 1h 2m 33s	remaining: 1h 40m 37s
12000:	test: 0.8955761	best: 0.8955761 (11990)	total: 1h 5m 17s	remaining: 1h 37m 55s
12500:	test: 0.8955811	best: 0.8955818 (12478)	total: 1h 7m 59s	remaining: 1h 35m 11s
13000:	test: 0.8955910	best: 0.8955910 (13000)	total: 1h 10m 44s	remaining: 1h 32m 29s
13500:	test: 0.8956019	best: 0.8956019 (13500)	total: 1h 13m 27s	remaining: 1h 29m 46s
14000:	test: 0.8956080	best: 0.8956081 (13995)	total: 1h 16m 12s	remaining: 1h 27m 5s
14500:	test: 0.8956116	best: 0.8956118 (14496)	total: 1h 18m 54s	remaining: 1h 24m 19s
15000:	test: 0.8956172	best: 0.8956173 (14992)	total: 1h 21m 37s	remaining: 1h 21m 36s
#### bestTest = 0.8956198692
#### bestIteration = 15268
#### Shrink model to first 15269 iterations.
#### Fold ROC-AUC score:  0.8956198692321777

# Total run time 14 hours, 55 minutes, and 47 seconds.

---
# Print the mean ROC-AUC score across all folds

In [ ]:
print('Mean ROC-AUC score: ', sum(scores) / FOLDS)

Mean ROC-AUC score:  0.8956927210092545

---
# Process the test set predictions by averaging them across all folds, then prepare the final submission file:
- Load the sample submission CSV to use as a template.
- Assign the averaged predictions to the target column in the submission file.
- Ensure the 'id' column is properly formatted as integers.
- Save the completed submission file as a new CSV.
- Display the submission DataFrame for verification.

In [ ]:
test_preds = np.mean(test_preds, axis=1)
submission = pd.read_csv('sample_submission.csv')
submission[target] = test_preds.astype(np.float32)
submission['id'] = submission['id'].astype(np.int32)
submission.to_csv('submission.csv', index=False)

---

# Our final Public Score was 0.89747 (89.747%) putting us in 9th place where we were 0.042% from first place.
---